In [2]:
import torch
# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(7, 5)

In [3]:
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [4]:
from unsloth import FastLanguageModel
import torch 
max_seq_length = 2048
dtype = None
load_in_4bit = True

model,tokenizer =  FastLanguageModel.from_pretrained(
    model_name='gemma-2b-jeonse_fraud',
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit = load_in_4bit
)

/home/ubuntu/miniconda3/envs/gemma2_env310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post2: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.9.post2 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [7]:

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
EOS_TOKEN

'<eos>'

In [8]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [9]:
# 데이터셋 만들어서 대체하기

from datasets import load_dataset
dataset = load_dataset("ball0428/qa_pair_judicial_precedent_data", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [10]:
dataset.features

{'instruction': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2): 100%|██████████| 963/963 [00:01<00:00, 533.28 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [12]:
dataset[132]['instruction']

'원고와 피고를 비롯한 구 건물의 공유자들 사이에 신 건물이 건축된 후 각자의 점유부분에 상응하는 부분을 소유하기로 한 약정이 있었음에도 불구하고, 원고가 신 건물 2층의 소유권을 원시취득하지 못한 이유는 무엇인가요?'

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 963 | Num Epochs = 9
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,3.198900
2,2.738700
3,2.512700
4,2.886800
5,1.986500
6,1.914000
7,1.651800
8,1.536600
9,1.495000
10,1.366900


In [23]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "국토교통부장관은 전세사기피해자등 결정을 위해 어떤 절차를 따라야 하나요?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
국토교통부장관은 전세사기피해자등 결정을 위해 어떤 절차를 따라야 하나요?

### Input:


### Response:
국토교통부장관은 유치관광사업에 해당하여야 하나인지 여부를 심사하고 결정하기 위해, 전세사기피해자등 결정에 필요한 자료를 정하여야 합니다.<eos>


In [14]:
model.save_pretrained("gemma-2b-jeonse_fraud_add_judicial") # Local saving
tokenizer.save_pretrained("gemma-2b-jeonse_fraud_add_judicial")

('gemma-2b-jeonse_fraud_add_judicial/tokenizer_config.json',
 'gemma-2b-jeonse_fraud_add_judicial/special_tokens_map.json',
 'gemma-2b-jeonse_fraud_add_judicial/tokenizer.model',
 'gemma-2b-jeonse_fraud_add_judicial/added_tokens.json',
 'gemma-2b-jeonse_fraud_add_judicial/tokenizer.json')

In [37]:

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "gemma-2b-judicial_precedent_data", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "전세유사의 계약에서 피고가 가옥을 계속 사용할 수 있는 조건은 무엇인가요?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

==((====))==  Unsloth 2024.9.post2: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.581 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n전세유사의 계약에서 피고가 가옥을 계속 사용할 수 있는 조건은 무엇인가요?\n\n### Input:\n\n\n### Response:\n피고는 원고로부터 금 1,000,000원을 지급받으면 가옥을 반환하여 원상태로 restituere에 부수하여 원고의 명도 없이 계속 사용할 수 있습니다.<eos>']

In [24]:
# 모델 저장(위 두개는 어댑터랑 베이스 모델 합해서 저장하는 것)

# Merge to 16bit
if True: model.save_pretrained_merged("ball0428/gemma-merge", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("ball0428/gemma-2b-jeonse_fraud", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("judicial_precedent_data", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("ball0428/gemma-2b-jeonse_fraud", tokenizer, save_method = "lora", token = "")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 15.39 out of 30.89 RAM for saving.


100%|██████████| 26/26 [00:00<00:00, 38.86it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [26]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if True: model.push_to_hub_gguf("ball0428/gemma-merge", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 17.22 out of 30.89 RAM for saving.


100%|██████████| 26/26 [00:00<00:00, 40.43it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at ball0428/gemma-merge into q8_0 GGUF format.
The output location will be ./ball0428/gemma-merge/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: gemma-merge
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00002.safetensors'
INFO:hf-

unsloth.Q8_0.gguf: 100%|██████████| 2.78G/2.78G [03:14<00:00, 14.3MB/s]


Saved GGUF to https://huggingface.co/ball0428/gemma-merge


In [1]:
# 병합된 모델을 불러와서 사용해보기
from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

/home/ubuntu/miniconda3/envs/gemma2_env310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.05s/it]


In [27]:
import torch
# 모델을 GPU로 이동 (가능한 경우)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 테스트용 입력 문장 생성
input_text = "What is a famous tall tower in Paris?"
inputs = tokenizer(
[
    alpaca_prompt.format(
        "3원색이란 무엇인가요?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# 모델을 사용하여 문장 생성
outputs = model.generate(**inputs, max_new_tokens=128)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
3원색이란 무엇인가요?

### Input:


### Response:
3원색은 3개의 색상을 사용하여 색상을 표현하는 색상 모델입니다. 3원색은 1931년 찰스 컬러와 1934년 찰스 컬러를 발명한 찰스 컬러가 개발한 색상 모델입니다. 3원색은 3개의 색상을 사용하여 색상을 표현하는 데 사용되는 색상 모델입니다. 3원색은 3개의 색상을 사용하여 색상을 표현하는 데 사용되는 색상 모델입니다.
